In [3]:
from simtk.openmm import XmlSerializer

from simtk import unit, openmm
# from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
# from perses.dispersed.utils import configure_platform
import numpy as np
import copy
import pickle

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
DEFAULT_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


In [37]:
with open("hybrid_system.xml", "r") as f:
    system = XmlSerializer.deserialize(f.read())

with open("hybrid_positions.pickle", "rb") as f:
    positions = pickle.load(f)

In [64]:
with open("old_system.xml", "r") as f:
    system = XmlSerializer.deserialize(f.read())

with open("old_positions.pickle", "rb") as f:
    positions = pickle.load(f)

In [3]:
def compute_potential_components2(system, positions, beta=beta, platform=DEFAULT_PLATFORM):
    """
    Compute potential energy, raising an exception if it is not finite.
    Parameters
    ----------
    context : simtk.openmm.Context
        The context from which to extract, System, parameters, and positions.
    """
    # Make a deep copy of the system.
    import copy

    from perses.dispersed.utils import configure_platform
    platform = configure_platform(platform.getName(), fallback_platform_name='Reference', precision='double')

#     system = context.getSystem()
#     system = copy.deepcopy(system)
    # Get positions.
#     positions = context.getState(getPositions=True).getPositions(asNumpy=True)
    # Get Parameters
#     parameters = context.getParameters()
    # Segregate forces.
    for index in range(system.getNumForces()):
        force = system.getForce(index)
        print("setting force: ", force.__class__.__name__, index)
        force.setForceGroup(31)
        if force.__class__.__name__ == 'NonbondedForce':
            force.setReciprocalSpaceForceGroup(31)
    # Create new Context.
    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator, platform)
    context.setPositions(positions)
    for (parameter, value) in context.getParameters().items():
        print(parameter, value)
#         context.setParameter(parameter, value)
    energy_components = list()
    for index in range(system.getNumForces()):
        force = system.getForce(index)
        forcename = force.__class__.__name__
        print(index, forcename, force.getForceGroup())
        if forcename == 'NonbondedForce':
            print("reciprocal space ", force.getReciprocalSpaceForceGroup())
#         groups = 1<<index
        groups = set([index])
        potential = beta * context.getState(getEnergy=True, groups=groups).getPotentialEnergy()
        energy_components.append((forcename, potential))
    del context, integrator
    return energy_components

In [69]:
# system.removeForce(7)

In [25]:
# system.getForce(7).setIncludeDirectSpace(True)

In [72]:
system.getForce(7).getUseDispersionCorrection()

True

In [7]:
for i in range(system.getForce(7).getNumParticles()):
    charge, sigma, epsilon = system.getForce(7).getParticleParameters(i)
#     if i > 21:
#         system.getForce(3).setParticleParameters(i, charge*0, sigma, epsilon*0)
#     else:
    system.getForce(7).setParticleParameters(i, charge*0, sigma, epsilon)



In [38]:
for i in range(system.getForce(7).getNumParticleParameterOffsets()):
    parameter, idx, chargeScale, sigmaScale, epsilonScale = system.getForce(7).getParticleParameterOffset(i)
    system.getForce(7).setParticleParameterOffset(i, parameter, idx, 0.0, 0.0, 0.0)

In [58]:
for i in range(system.getForce(7).getNumExceptions()):
    p1, p2, charge, sigma, epsilon = system.getForce(7).getExceptionParameters(i)
#     if p1 > 21 or p2 > 21:
#         system.getForce(3).setExceptionParameters(i, p1, p2, charge*0, sigma, epsilon*0)
#     else:
    system.getForce(7).setExceptionParameters(i, p1, p2, charge, sigma, epsilon)
    

In [39]:
compute_potential_components2(system, positions)

conducting subsequent work with the following platform: CUDA
setting force:  MonteCarloBarostat 0
setting force:  CustomBondForce 1
setting force:  CustomAngleForce 2
setting force:  CustomTorsionForce 3
setting force:  CustomNonbondedForce 4
setting force:  CustomNonbondedForce 5
setting force:  CustomBondForce 6
setting force:  NonbondedForce 7
MonteCarloPressure 1.01325
MonteCarloTemperature 300.0
lambda_alchemical_angles_new 0.0
lambda_alchemical_angles_old 1.0
lambda_alchemical_bonds_new 0.0
lambda_alchemical_bonds_old 1.0
lambda_alchemical_electrostatics_exceptions_new 0.0
lambda_alchemical_electrostatics_exceptions_old 1.0
lambda_alchemical_electrostatics_new 0.0
lambda_alchemical_electrostatics_old 1.0
lambda_alchemical_electrostatics_reciprocal 0.0
lambda_alchemical_sterics_exceptions_new 0.0
lambda_alchemical_sterics_exceptions_old 1.0
lambda_alchemical_sterics_new 0.0
lambda_alchemical_sterics_old 1.0
lambda_alchemical_torsions_new 0.0
lambda_alchemical_torsions_old 1.0
lamb

[('MonteCarloBarostat', -49274.68888855932),
 ('CustomBondForce', -49274.68888855932),
 ('CustomAngleForce', -49274.68888855932),
 ('CustomTorsionForce', -49274.68888855932),
 ('CustomNonbondedForce', -49274.68888855932),
 ('CustomNonbondedForce', -49274.68888855932),
 ('CustomBondForce', -49274.68888855932),
 ('NonbondedForce', -49274.68888855932)]

# Play around with building the NonbondedForce

In [14]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [16]:
htf._topology_proposal.old_system.getForce(3).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

In [33]:
hybrid_system = openmm.System()

for particle_idx in range(htf._topology_proposal.n_atoms_old):
    particle_mass = htf._old_system.getParticleMass(particle_idx)
    hybrid_idx = hybrid_system.addParticle(particle_mass)

# Next, add the remaining unique atoms from the new system to the hybrid system and map accordingly.
# As before, this does not copy interactions, only particle indices and masses.
for particle_idx in htf._topology_proposal.unique_new_atoms:
    particle_mass = htf._new_system.getParticleMass(particle_idx)
    hybrid_idx = hybrid_system.addParticle(particle_mass)


In [34]:

# Create force
standard_nonbonded_force = openmm.NonbondedForce()
hybrid_system.addForce(standard_nonbonded_force)

# Add global parameters
standard_nonbonded_force.addGlobalParameter("lambda_alchemical_electrostatics_reciprocal", 0.0)

# Set nonbonded method and related attributes
old_system_nbf = htf._topology_proposal.old_system.getForce(3)
standard_nonbonded_force.setNonbondedMethod(old_system_nbf.getNonbondedMethod())
nonbonded_method = standard_nonbonded_force.getNonbondedMethod() 
if nonbonded_method in [openmm.NonbondedForce.CutoffPeriodic, openmm.NonbondedForce.CutoffNonPeriodic]:
    standard_nonbonded_force.setReactionFieldDielectric(old_system_nbf.getReactionFieldDielectric())
    standard_nonbonded_force.setCutoffDistance(1.0*unit.nanometer)
elif nonbonded_method in [openmm.NonbondedForce.PME, openmm.NonbondedForce.Ewald]:
    [alpha_ewald, nx, ny, nz] = old_system_nbf.getPMEParameters()
    delta = old_system_nbf.getEwaldErrorTolerance()
    standard_nonbonded_force.setPMEParameters(alpha_ewald, nx, ny, nz)
    standard_nonbonded_force.setEwaldErrorTolerance(delta)
    standard_nonbonded_force.setCutoffDistance(1.0*unit.nanometer)
elif nonbonded_method in [openmm.NonbondedForce.NoCutoff]:
    pass
else:
    raise Exception("Nonbonded method %s not supported yet." % str(nonbonded_method))

# Set the use of dispersion correction
if old_system_nbf.getUseDispersionCorrection():
    standard_nonbonded_force.setUseDispersionCorrection(True)
else:
    standard_nonbonded_force.setUseDispersionCorrection(False)

# Set the use of switching function
if old_system_nbf.getUseSwitchingFunction():
    standard_nonbonded_force.setUseSwitchingFunction(True)
    standard_nonbonded_force.setSwitchingDistance(old_system_nbf.getSwitchingDistance())
else:
    standard_nonbonded_force.setUseSwitchingFunction(False)

# Disable direct space interactions
standard_nonbonded_force.setIncludeDirectSpace(False)

In [35]:
# Retrieve old and new nb forces
old_system_nbf = htf._topology_proposal.old_system.getForce(3)
new_system_nbf = htf._topology_proposal.new_system.getForce(3)

# Retrieve atom index maps from hybrid to old/new
hybrid_to_old_map = htf._hybrid_to_old_map
hybrid_to_new_map = htf._hybrid_to_new_map

# Retrieve atom classes
atom_classes = htf._atom_classes

# Iterate over the particles in the hybrid system, because nonbonded force does not accept index
for particle_index in range(htf._hybrid_system.getNumParticles()):

    if particle_index in atom_classes['unique_old_atoms']:
        # Get the parameters in the old system
        old_index = hybrid_to_old_map[particle_index]
        charge, sigma, epsilon = old_system_nbf.getParticleParameters(old_index)

        # Add particle to the standard nonbonded force
        check_index = standard_nonbonded_force.addParticle(charge, sigma * 0.0, epsilon * 0.0)
        assert (particle_index == check_index), "Attempting to add incorrect particle to hybrid system"

#         # Charge and epsilon will be turned on at lambda = 0 and turned off at lambda = 1
#         standard_nonbonded_force.addParticleParameterOffset('lambda_alchemical_electrostatics_reciprocal', particle_index, -charge, 0.0, 0.0)

    elif particle_index in atom_classes['unique_new_atoms']:
        # Get the parameters in the new system
        new_index = hybrid_to_new_map[particle_index]
        charge, sigma, epsilon = new_system_nbf.getParticleParameters(new_index)

        # Add particle to the standard nonbonded force
        check_index = standard_nonbonded_force.addParticle(charge * 0.0, sigma * 0.0, epsilon * 0.0) # charge and epsilon start at zero
        assert (particle_index == check_index), "Attempting to add incorrect particle to hybrid system"

#         # Charge and epsilon will be turned off at lambda = 0 and turned on at lambda = 1
#         standard_nonbonded_force.addParticleParameterOffset('lambda_alchemical_electrostatics_reciprocal', particle_index, charge, 0.0, 0.0)

    elif particle_index in atom_classes['core_atoms']:
        # Get the parameters in the new and old systems:
        old_index = hybrid_to_old_map[particle_index]
        charge_old, sigma_old, epsilon_old = old_system_nbf.getParticleParameters(old_index)
        new_index = hybrid_to_new_map[particle_index]
        charge_new, sigma_new, epsilon_new = new_system_nbf.getParticleParameters(new_index)

        # Add the particle to the standard nonbonded force
        check_index = standard_nonbonded_force.addParticle(charge_old, (sigma_old + sigma_new) * 0.0, epsilon_old * 0.0)
        assert (particle_index == check_index), "Attempting to add incorrect particle to hybrid system"

#         # Charge is charge_old at lambda_electrostatics = 0, charge_new at lambda_electrostatics = 1
#         # TODO: We could also interpolate the Lennard-Jones here instead of core_sterics force so that core_sterics_force could just be softcore
#         # interpolate between old and new charge with lambda_electrostatics core; make sure to keep sterics off
#         standard_nonbonded_force.addParticleParameterOffset('lambda_alchemical_electrostatics_reciprocal', particle_index, (charge_new - charge_old), 0.0, 0.0)

    # Otherwise, the particle is in the environment
    else:
        # The parameters will be the same in new and old system, so just take the old parameters
        old_index = hybrid_to_old_map[particle_index]
        charge_old, sigma_old, epsilon_old = old_system_nbf.getParticleParameters(old_index)
        new_index = hybrid_to_new_map[particle_index]
        charge_new, sigma_new, epsilon_new = new_system_nbf.getParticleParameters(new_index)
        assert charge_old == charge_new, "charges do not match: {charge_old} (old) and {charge_new} (new)"
        assert sigma_old == sigma_new, f"sigmas do not match: {sigma_old} (old) and {sigma_new} (new)"
        assert epsilon_old == epsilon_new, f"epsilons do not match: {epsilon_old} (old) and {epsilon_new} (new)"

        # Add the environment atoms to the stanard nonbonded force
        standard_nonbonded_force.addParticle(charge_old, sigma_old * 0.0, epsilon_old * 0.0)

# # Add exceptions for unique/old so that they never interact
# for old in atom_classes['unique_old_atoms']:
#     for new in atom_classes['unique_new_atoms']:
#         standard_nonbonded_force.addException(old,
#                                               new,
#                                               0.0 * unit.elementary_charge ** 2,
#                                               1.0 * unit.nanometers,
#                                               0.0 * unit.kilojoules_per_mole)

In [36]:
compute_potential_components2(hybrid_system, htf.hybrid_positions)

conducting subsequent work with the following platform: CUDA
setting force:  NonbondedForce 0
lambda_alchemical_electrostatics_reciprocal 0.0
0 NonbondedForce 31
reciprocal space  31


[('NonbondedForce', 0.0)]

# Make simpler system to show problem

In [48]:
from openmmtools.testsystems import AlanineDipeptideExplicit

In [103]:
atp = AlanineDipeptideExplicit()

In [104]:
system = openmm.System()

for particle_idx in range(atp.system.getNumParticles()):
    particle_mass = atp.system.getParticleMass(particle_idx)
    system.addParticle(particle_mass)


In [105]:
# Create force
standard_nonbonded_force = openmm.NonbondedForce()
system.addForce(standard_nonbonded_force)

# Add global parameters
standard_nonbonded_force.addGlobalParameter("lambda_alchemical_electrostatics_reciprocal", 0.0)

# old_nbf = atp.system.getForce(3)
standard_nonbonded_force.setNonbondedMethod(old_nbf.getNonbondedMethod())
# [alpha_ewald, nx, ny, nz] = old_nbf.getPMEParameters()
# delta = old_nbf.getEwaldErrorTolerance()
# standard_nonbonded_force.setPMEParameters(alpha_ewald, nx, ny, nz)
# standard_nonbonded_force.setEwaldErrorTolerance(delta)
# standard_nonbonded_force.setCutoffDistance(1.0*unit.nanometer)

# # Disable direct space interactions
# standard_nonbonded_force.setIncludeDirectSpace(False)

In [107]:
# Iterate over the particles in the hybrid system, because nonbonded force does not accept index
for particle_index in range(system.getNumParticles()):

    charge, sigma, epsilon = atp.system.getForce(3).getParticleParameters(particle_index)

    # Add particle to the standard nonbonded force
    standard_nonbonded_force.addParticle(charge, sigma * 0.0, epsilon * 0.0)

#     # Charge and epsilon will be turned on at lambda = 0 and turned off at lambda = 1
#     standard_nonbonded_force.addParticleParameterOffset('lambda_alchemical_electrostatics_reciprocal', particle_index, -charge, 0.0, 0.0)



In [108]:
compute_potential_components2(system, atp.positions)

conducting subsequent work with the following platform: CUDA
setting force:  NonbondedForce 0
lambda_alchemical_electrostatics_reciprocal 0.0
0 NonbondedForce 31
reciprocal space  31


[('NonbondedForce', 0.0)]

In [1]:
from openmmtools.testsystems import AlanineDipeptideExplicit
from simtk import unit, openmm
from openmmtools.constants import kB

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
DEFAULT_PLATFORM = openmm.Platform.getPlatformByName("CUDA")

# Grab reference system
atp = AlanineDipeptideExplicit()

# Create system from scratch
system = openmm.System()

# Copy particles from reference to new system
for particle_idx in range(atp.system.getNumParticles()):
    particle_mass = atp.system.getParticleMass(particle_idx)
    system.addParticle(particle_mass)

# Create nonbonded force
standard_nonbonded_force = openmm.NonbondedForce()
system.addForce(standard_nonbonded_force)

# Add global parameters
standard_nonbonded_force.addGlobalParameter("lambda_alchemical_electrostatics_reciprocal", 0.0)

# Set nonbonded method
old_nbf = atp.system.getForce(3)
standard_nonbonded_force.setNonbondedMethod(old_nbf.getNonbondedMethod())
# standard_nonbonded_force.setNonbondedMethod(openmm.NonbondedForce.CutoffPeriodic)
# print(openmm.NonbondedForce.CutoffPeriodic)
# standard_nonbonded_force.setUseDispersionCorrection(False)
print(standard_nonbonded_force.getNonbondedMethod())
print(standard_nonbonded_force.getUseDispersionCorrection())
# standard_nonbonded_force.setReciprocalSpaceForceGroup(1)

# Copy particles from reference system to new system's nonbonded force
for particle_index in range(system.getNumParticles()):

    charge, sigma, epsilon = old_nbf.getParticleParameters(particle_index)

    # Add particle to the standard nonbonded force
    standard_nonbonded_force.addParticle(charge, sigma * 0.0, epsilon * 0.0)

    # Charge and epsilon will be turned on at lambda = 0 and turned off at lambda = 1
    standard_nonbonded_force.addParticleParameterOffset('lambda_alchemical_electrostatics_reciprocal', particle_index, -charge, 0.0, 0.0)

def compute_potential_components(system, positions, beta=beta, platform=DEFAULT_PLATFORM):
    """
    Compute potential energy, raising an exception if it is not finite.
    Parameters
    ----------
    context : simtk.openmm.Context
        The context from which to extract, System, parameters, and positions.
    """
 
#     from perses.dispersed.utils import configure_platform
#     platform = configure_platform(platform.getName(), fallback_platform_name='Reference', precision='double')
    platform = openmm.Platform.getPlatformByName('CUDA')
    platform.setPropertyDefaultValue('Precision', 'double')

    # Set force group to 31
    force = system.getForce(0)
    force.setForceGroup(31)

    # Create new Context.
    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator, platform)
    context.setPositions(positions)
    
    # Retrieve energy for force group 0
    groups = set([0])
    potential = beta * context.getState(getEnergy=True, groups=groups).getPotentialEnergy()
    print(potential)
#     groups = set([1])
#     potential = beta * context.getState(getEnergy=True, groups=groups).getPotentialEnergy()
#     print(potential)
#     groups = set([31])
#     potential = beta * context.getState(getEnergy=True, groups=groups).getPotentialEnergy()
#     print(potential)
    potential = beta * context.getState(getEnergy=True, groups=groups).getPotentialEnergy()
    print(potential)
    del context, integrator
    
    

compute_potential_components(system, atp.positions)

4
True
0.0
0.0


In [19]:
with open('system_with_offset.xml', 'w') as f:
    f.write(XmlSerializer.serialize(system))

In [23]:
with open('system_without_offset.xml', 'w') as f:
    f.write(XmlSerializer.serialize(system))

In [25]:
import pickle

In [26]:
with open("positions.pickle", "wb") as f:
    pickle.dump(atp.positions, f)